In [79]:
import requests
from lxml import etree
import urllib.parse
import crawler
from bs4 import BeautifulSoup
import requests
import csv
import time
import urllib.request
import re
import json
import datetime
import pandas as pd
import numpy as np

In [128]:
#today = datetime.date.today().strftime('%Y%m%d')   
def crawl_wiki_data(url,title):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.57'}
    try:
        response = requests.get(url,headers=headers)
        response.raise_for_status()
        response.encoding = 'utf-8'

        #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
        soup = BeautifulSoup(response.text,'lxml')

        #返回的是class为table-view log-set-param的<table>所有标签
        tables = soup.find_all('table',{"log-set-param":"table_view"})
        #print("tables:",tables)
        if title=='马德里竞技足球俱乐部'or'巴塞罗那足球俱乐部':
            crawl_table_title1 = "球员列表"
        if title=='皇家马德里足球俱乐部'or'塞维利亚足球俱乐部'or'皇家贝蒂斯足球俱乐部'or'比利亚雷亚尔足球俱乐部'or'毕尔巴鄂竞技俱乐部'or'瓦伦西亚足球俱乐部'or '奥萨苏纳竞技俱乐部'or'皇家西班牙人足球俱乐部'or'皇家比戈塞尔塔足球俱乐部'or'马德里巴列卡诺闪电足球俱乐部'or'埃尔切足球俱乐部'or'赫塔费足球俱乐部'or'皇家马略卡足球俱乐部'or'格拉纳达足球俱乐部'or'阿拉维斯足球俱乐部'or'莱万特足球俱乐部':
            crawl_table_title2 = "球队阵容"
            
        for table in tables:           
            #对当前节点前面的标签和字符串进行查找
            
            table_titles = table.find_previous('div').find_all('h2')
            
            for title_temp in table_titles:
                if(crawl_table_title1 in title_temp):
                    print("找到特定的table1了")#,table)
                    return table    
                if(crawl_table_title2 in title_temp):
                    print("找到特定的table2了")#,table)
                    return table    
    except Exception as e:
        print("获得网页失败")

In [129]:
def parse_wiki_data(table_html):
    '''
    从百度百科返回的html中解析得到选手信息，以当前日期作为文件名，存JSON文件,保存到work目录下
    '''
    bs = BeautifulSoup(str(table_html),'lxml')
    all_trs = bs.find_all('tr')
    #print("all_trs:\n",all_trs)

    error_list = ['\'','\"']

    stars = []

    for tr in all_trs[1:]:
        all_tds = tr.find_all('td')
        #print("all_tds:\n",all_tds)

        star = {}

        #号码
        star["number"]=all_tds[0].text
        #个人百度百科链接
        #star["link"]= 'https://baike.baidu.com' + all_tds[0].find('a').get('href')
        #姓名
        star["name"]=all_tds[1].text
        #国籍
        star["country"]=all_tds[2].text
        #生日
        star["birthday"]=all_tds[3].text
        #场上位置
        star["location"]= all_tds[4].text
        stars.append(star)
        #print("stars:",stars)
    return stars

In [99]:
def getHTMLText(url):
    '''
    获取网页内容
    '''
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.57'}
        r = requests.get(url,headers=headers)
        #r=urllib.request.urlopen(url)
        r.raise_for_status()
        r.encoding = 'utf-8'
        #print("r.apparent_encoding", r.apparent_encoding)
        #print("r.text:",r.text)
        return r.text
    except:
        print('无法获取网页内容！！', url)
        return
    
def getTitle(url, obj=''):
    '''
    获取页面infobox信息
    参数：url-页面链接；obj-页面标题
    返回值：属性键值对字典
    '''
    html_text = getHTMLText(url)
    #print(html_text)
    soup = BeautifulSoup(html_text, 'html.parser')
  
    title_node = soup.find('dd', class_='lemmaWgt-lemmaTitle-title J-lemma-title').find('h1')
    title = title_node.get_text()
    print(title_node.get_text())
    return title

def write_csv(title,stars):
    df = pd.DataFrame(stars)
    df.to_csv("players\ " + title +".csv",encoding ='gbk')

In [83]:
df = pd.DataFrame(stars)
df

,number,name,country,birthday,location
0,1,马克-安德烈·特尔施特根,德国,1992-04-30,门将
1,2,塞尔吉尼奥·德斯特,美国,2000-11-03,后卫
2,3,杰拉德·皮克,西班牙,1987-02-02,后卫
3,4,罗纳德·阿劳霍,乌拉圭,1999-03-07,后卫
4,5,塞尔吉奥·布斯克茨,西班牙,1988-07-16,中场
5,6,里基·普奇,西班牙,1999-08-13,中场
6,7,奥斯曼·登贝莱,法国,1997-05-15,前锋
7,8,达尼·阿尔维斯,巴西,1983-05-06,后卫
8,9,孟菲斯·德佩,荷兰,1994-02-13,前锋
9,10,安苏·法蒂,西班牙,2002-10-31,前锋


In [56]:
    #解析html,得到选手信息，保存为json文件
    stars=parse_wiki_data(html) 
    print("爬取完成")

爬取完成


In [94]:
url = 'https://baike.baidu.com/item/巴塞罗那足球俱乐部'
html = crawl_wiki_data(url)
title=getTitle(url)
stars=parse_wiki_data(html)
print(type(stars))
print(stars)
write_csv(title, stars)

找到特定的table了：

巴塞罗那足球俱乐部
<class 'list'>
[{'number': '1', 'name': '马克-安德烈·特尔施特根', 'country': '德国', 'birthday': '1992-04-30', 'location': '门将'}, {'number': '2', 'name': '塞尔吉尼奥·德斯特', 'country': '美国', 'birthday': '2000-11-03', 'location': '后卫'}, {'number': '3', 'name': '杰拉德·皮克', 'country': '西班牙', 'birthday': '1987-02-02', 'location': '后卫'}, {'number': '4', 'name': '罗纳德·阿劳霍', 'country': '乌拉圭', 'birthday': '1999-03-07', 'location': '后卫'}, {'number': '5', 'name': '塞尔吉奥·布斯克茨', 'country': '西班牙', 'birthday': '1988-07-16', 'location': '中场'}, {'number': '6', 'name': '里基·普奇', 'country': '西班牙', 'birthday': '1999-08-13', 'location': '中场'}, {'number': '7', 'name': '奥斯曼·登贝莱', 'country': '法国', 'birthday': '1997-05-15', 'location': '前锋'}, {'number': '8', 'name': '达尼·阿尔维斯', 'country': '巴西', 'birthday': '1983-05-06', 'location': '后卫'}, {'number': '9', 'name': '孟菲斯·德佩', 'country': '荷兰', 'birthday': '1994-02-13', 'location': '前锋'}, {'number': '10', 'name': '安苏·法蒂', 'country': '西班牙', 'birthday': '2002-10-31', 

In [101]:
def players(url):
    title=getTitle(url)
    table=crawl_wiki_data(url,title)
    stars=parse_wiki_data(table)
    write_csv(title, stars)

In [122]:
root='https://baike.baidu.com/item/'
url=['皇家马德里足球俱乐部','塞维利亚足球俱乐部','皇家贝蒂斯足球俱乐部', '马德里竞技足球俱乐部','巴塞罗那足球俱乐部',
     '比利亚雷亚尔足球俱乐部','毕尔巴鄂竞技俱乐部','瓦伦西亚足球俱乐部', '奥萨苏纳竞技俱乐部',
     '皇家西班牙人足球俱乐部','皇家比戈塞尔塔足球俱乐部','马德里巴列卡诺闪电足球俱乐部','埃尔切足球俱乐部','赫塔费足球俱乐部',
    '皇家马略卡足球俱乐部','格拉纳达足球俱乐部','阿拉维斯足球俱乐部','莱万特足球俱乐部']
for i in range(len(url)):
    temp=root+url[i]
    #print(temp)
    players(temp)

皇家马德里足球俱乐部
找到特定的table2了
塞维利亚足球俱乐部
找到特定的table2了
皇家贝蒂斯足球俱乐部
找到特定的table2了
马德里竞技足球俱乐部
找到特定的table1了
巴塞罗那足球俱乐部
找到特定的table1了
比利亚雷亚尔足球俱乐部
找到特定的table2了
毕尔巴鄂竞技足球俱乐部
找到特定的table2了
瓦伦西亚足球俱乐部
找到特定的table2了
奥萨苏纳足球俱乐部
找到特定的table2了
皇家西班牙人足球俱乐部
找到特定的table2了
皇家比戈塞尔塔足球俱乐部
找到特定的table2了
马德里巴列卡诺闪电足球俱乐部
找到特定的table2了
埃尔切足球俱乐部
找到特定的table2了
赫塔费足球俱乐部
找到特定的table2了
皇家马略卡足球俱乐部
找到特定的table2了
格拉纳达足球俱乐部
找到特定的table2了
阿拉维斯足球俱乐部
找到特定的table2了
莱万特足球俱乐部
找到特定的table2了


## 例外
加的斯足球俱乐部

In [131]:
#today = datetime.date.today().strftime('%Y%m%d')   
def crawl_wiki_data(url,title):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.57'}
    try:
        response = requests.get(url,headers=headers)
        response.raise_for_status()
        response.encoding = 'utf-8'

        #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
        soup = BeautifulSoup(response.text,'lxml')

        #返回的是class为table-view log-set-param的<table>所有标签
        tables = soup.find_all('table',{"log-set-param":"table_view"})
        #print("tables:",tables)
        crawl_table_title2 = "球队阵容"
            
        for table in tables:           
            #对当前节点前面的标签和字符串进行查找
            
            table_titles = table.find_previous('div').find_all('h2')
            
            for title_temp in table_titles:
                if(crawl_table_title2 in title_temp):
                    print("找到特定的table2了")#,table)
                    return table    
    except Exception as e:
        print("获得网页失败")

In [132]:
def parse_wiki_data(table_html):
    '''
    从百度百科返回的html中解析得到选手信息，以当前日期作为文件名，存JSON文件,保存到work目录下
    '''
    bs = BeautifulSoup(str(table_html),'lxml')
    all_trs = bs.find_all('tr')
    #print("all_trs:\n",all_trs)

    error_list = ['\'','\"']

    stars = []

    for tr in all_trs[1:26]:
        all_tds = tr.find_all('td')
        #print("all_tds:\n",all_tds)

        star = {}

        #号码
        star["number"]=all_tds[0].text
        #个人百度百科链接
        #star["link"]= 'https://baike.baidu.com' + all_tds[0].find('a').get('href')
        #姓名
        star["name"]=all_tds[1].text
        #国籍
        star["country"]=all_tds[5].text
        #生日
        star["birthday"]=all_tds[4].text
        #场上位置
        star["location"]= all_tds[3].text
        #英文名
        star["name-eng"]=all_tds[2].text
        stars.append(star)
        #print("stars:",stars)
    return stars

In [133]:
players("https://baike.baidu.com/item/加的斯足球俱乐部")

加的斯足球俱乐部
找到特定的table2了


皇家社会足球俱乐部

In [130]:
players("https://baike.baidu.com/item/皇家社会足球俱乐部/1531951")

皇家社会足球俱乐部
找到特定的table2了
